<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Вторая часть работы по оптимизации классификации заявок на оплату для сервиса Repetit.ru

В этой тетрадке произведены следующие действия:
- открыт и изучен файл с данными о подходящих преподавателях suitable_teachers.feather, произведена предобработка данных
- открыт и изучен файл с информацией об учителях teachers_info.feather, произведена предобработка данных
- файлы с информацие об учителях объединены и сохранены

In [1]:
#pip install --upgrade pandas

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time
import requests
from tqdm import tqdm
import numpy as np

# Откроем и изучим файл с данными о подходящих преподавателях (suitable_teachers.feather)

In [3]:
try:
    suit_t = pd.read_feather(r'C:\Users\User\Desktop\DS\Repetit\datasets\suitable_teachers.feather')
except:

    suit_t = pd.read_feather('suitable_teachers.feather')  

In [4]:
suit_t

,teacher_id,order_id,contact_result,enable_auto_assign,enable_assign
0,82583,1742711,Репетитор согласился,1,1
1,125057,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0,0
2,231211,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0,0
3,128773,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0,0
4,63019,1742713,Репетитор согласился,0,1
...,...,...,...,...,...
20271181,292878,2946697,NaN,0,0
20271182,36028,2946079,NaN,0,0
20271183,180592,2946079,NaN,0,0
20271184,87326,2946079,NaN,0,0


In [5]:
suit_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20271186 entries, 0 to 20271185
Data columns (total 5 columns):
 #   Column              Dtype   
---  ------              -----   
 0   teacher_id          int32   
 1   order_id            int32   
 2   contact_result      category
 3   enable_auto_assign  int8    
 4   enable_assign       int8    
dtypes: category(1), int32(2), int8(2)
memory usage: 232.3 MB


In [6]:
suit_t.isna().sum()

teacher_id                   0
order_id                     0
contact_result        12824362
enable_auto_assign           0
enable_assign                0
dtype: int64

## Удалим строки, где в столбце contact_result пропукси данных, а также полные дубликаты строк

In [7]:
suit_t=suit_t.dropna(subset=['contact_result'])

In [8]:
suit_t.duplicated().sum()

6

In [9]:
suit_t = suit_t.drop_duplicates().reset_index(drop=True)
suit_t

,teacher_id,order_id,contact_result,enable_auto_assign,enable_assign
0,82583,1742711,Репетитор согласился,1,1
1,125057,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0,0
2,231211,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0,0
3,128773,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0,0
4,63019,1742713,Репетитор согласился,0,1
...,...,...,...,...,...
7446813,175947,2944965,Репетитор согласился,1,1
7446814,197299,2946576,",",0,0
7446815,152334,2946689,Репетитор согласился,0,1
7446816,139143,2946689,Репетитор согласился,0,1


In [10]:
tqdm.pandas(desc="progress")

In [11]:
suit_t.rename(columns={'teacher_id':'working_teacher_id'}, inplace = True )

In [12]:
suit_t.rename(columns={'order_id':'id'}, inplace = True )

### Объединим колонки enable_auto_assign и enable_assign

In [13]:
suit_t['assign'] = suit_t['enable_auto_assign'] + suit_t['enable_assign']

In [14]:
suit_t = suit_t.drop(columns=['enable_auto_assign', 'enable_assign']) 

In [15]:
suit_t

,working_teacher_id,id,contact_result,assign
0,82583,1742711,Репетитор согласился,2
1,125057,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
2,231211,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
3,128773,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
4,63019,1742713,Репетитор согласился,1
...,...,...,...,...
7446813,175947,2944965,Репетитор согласился,2
7446814,197299,2946576,",",0
7446815,152334,2946689,Репетитор согласился,1
7446816,139143,2946689,Репетитор согласился,1


# Откроем и изучим таблицу с информацией об учителях teachers_info.feather

In [16]:
try:
    t_info = pd.read_feather(r'C:\Users\User\Desktop\DS\Repetit\datasets\teachers_info.feather')
except:

    t_info = pd.read_feather('teachers_info.feather')  

In [17]:
t_info.head(3)

,id,date_update,reg_date,birth_date,teaching_start_date,user_id,is_email_confirmed,is_home_lessons,is_external_lessons,external_comments,...,verification_status_id,is_individual,partner_id,star_rating,rating_for_users_yesterday,review_num,relevance_date,is_display_at_partners,status_institution_id,free_time_relevance_date
0,33,2023-11-07 14:20:28.323,2007-12-24 19:26:46.000,1983-06-05 00:00:00.000,1989-09-01 00:00:00.000,45,0,0,0,NaN,...,1,NaN,NaN,4.500000,1138.0,6,2022-05-16 18:44:59.993,0,3702.0,2023-11-06 14:20:15.173
1,46,2023-01-19 17:36:42.233,2007-12-24 19:26:46.000,1980-02-12 00:00:00.000,2003-09-01 00:00:00.000,63,0,1,1,NaN,...,1,NaN,NaN,4.300781,25.0,0,2017-01-01 00:00:00.000,1,NaN,NaN
2,51,2023-05-25 09:04:45.243,2007-12-24 19:26:46.000,1965-04-30 00:00:00.000,1988-09-01 00:00:00.000,68,0,0,0,NaN,...,3,0.0,NaN,4.300781,300.0,0,2017-01-01 00:00:00.000,0,NaN,NaN


In [18]:
#t_info.info()

## Приведем тип данных с информациями о датах к соответвующему

In [19]:
oneday = pd.Timedelta(days=365)

In [20]:
t_info['date_update'] = t_info['date_update'].astype('datetime64[ns]')
t_info['birth_date'] = t_info['birth_date'].astype('datetime64[ns]')
t_info['teaching_start_date'] = t_info['teaching_start_date'].astype('datetime64[ns]')

## Посчитаем возраст и опыт преподавателей

In [21]:
t_info['age'] = (t_info['date_update'] - t_info['birth_date'])/oneday
t_info['expirience'] = (t_info['date_update'] - t_info['teaching_start_date'])/oneday

In [22]:
t_info1 = t_info.drop(columns=['date_update', 'reg_date', 'birth_date', 'teaching_start_date', 'is_email_confirmed', 
                               'status_relevant_date', 'id', 'rules_confirmed_date', 'last_visited'], axis=1)

In [23]:
t_info1.head(2)

,user_id,is_home_lessons,is_external_lessons,external_comments,lesson_duration,lesson_cost,status_id,status_school_id,status_college_id,status_display,...,partner_id,star_rating,rating_for_users_yesterday,review_num,relevance_date,is_display_at_partners,status_institution_id,free_time_relevance_date,age,expirience
0,45,0,0,NaN,60,1000,4,NaN,NaN,0,...,NaN,4.500000,1138.0,6,2022-05-16 18:44:59.993,0,3702.0,2023-11-06 14:20:15.173,40.453692,34.207117
1,63,1,1,NaN,60,500,5,NaN,NaN,1,...,NaN,4.300781,25.0,0,2017-01-01 00:00:00.000,1,NaN,NaN,42.966394,19.399271


In [24]:
t_info1['external_comments'].unique()

[NaN, 'Ближайший район: м. Алтуфьево. ', 'Выезд в удобное для Вас время по Москве.', 'Ближайший район: м. Академическая. ', 'Ближайший район: м. Дмитровская. По договорен..., ..., 'Ближайший район: г. Белый Яр. ', 'Ближайший район: Речица микрорайон. ', 'Ближайший район: г. Таганрог. Выезд: город Та..., 'Ближайший район: м. Народное Ополчение. ', 'Ближайший район: Промышленный округ. ']
Length: 26083
Categories (26082, object): ['\r\n', '\r\n\r\n\r\n', ' ', '   Могу выезжать к ученикам по серой ветке мо..., ..., 'южное, северное бутово, метро аннино - южная', 'южный округ', 'юзао, зао', 'ясенево, теплый стан, коньково, беляево, калу...]

In [25]:
#t_info1 = t_info1.add_suffix('_teacher')

In [26]:
t_info1.head(5)

,user_id,is_home_lessons,is_external_lessons,external_comments,lesson_duration,lesson_cost,status_id,status_school_id,status_college_id,status_display,...,partner_id,star_rating,rating_for_users_yesterday,review_num,relevance_date,is_display_at_partners,status_institution_id,free_time_relevance_date,age,expirience
0,45,0,0,NaN,60,1000,4,NaN,NaN,0,...,NaN,4.500000,1138.0,6,2022-05-16 18:44:59.993,0,3702.0,2023-11-06 14:20:15.173,40.453692,34.207117
1,63,1,1,NaN,60,500,5,NaN,NaN,1,...,NaN,4.300781,25.0,0,2017-01-01 00:00:00.000,1,NaN,NaN,42.966394,19.399271
2,68,0,0,NaN,60,1500,5,NaN,NaN,1,...,NaN,4.300781,300.0,0,2017-01-01 00:00:00.000,0,NaN,NaN,58.107886,34.751721
3,69,0,1,Ближайший район: м. Алтуфьево.,60,1000,5,NaN,NaN,1,...,NaN,5.000000,135.0,2,2020-04-01 01:07:44.157,1,NaN,NaN,34.562860,16.395737
4,70,0,1,Выезд в удобное для Вас время по Москве.,45,400,5,NaN,NaN,1,...,NaN,4.300781,50.0,0,2017-01-01 00:00:00.000,1,NaN,NaN,37.330648,19.064895


In [27]:
t_info1.rename(columns={'user_id':'working_teacher_id'}, inplace = True )

## Удалим одинаковые колонки

In [28]:
t_info1['status_relevant_date.1'].equals(t_info1['status_relevant_date.2'])

True

In [29]:
t_info1 = t_info1.drop(columns=['status_relevant_date.2'], axis=1)  

In [30]:
def teacher_comparison(data):
    teacher = ['external_comments', 'lesson_duration', 'lesson_cost', 'status_id', 
            'status_school_id', 'status_college_id', 'status_display', 'russian_level_id', 'home_country_id', 
            'information', 'is_confirmed']
    list_of_statement = []
    #teacher = ['lesson_duration']
    for i in teacher:
        #list_of_statement.append(data[i].equals(data[i+'.1']))
        if data[i].equals(data[i+'.1']):
            data = data.drop(columns=[i+'.1'], axis=1)
            print(i+'.1 - deleted')
        if data[i].equals(data[i+'.2']):
            data = data.drop(columns=[i+'.2'], axis=1)   
            print(i+'.2 - deleted')
    return data

In [31]:
t_info2 = teacher_comparison(t_info1)

external_comments.1 - deleted
external_comments.2 - deleted
lesson_duration.1 - deleted
lesson_duration.2 - deleted
lesson_cost.1 - deleted
lesson_cost.2 - deleted
status_id.1 - deleted
status_id.2 - deleted
status_school_id.1 - deleted
status_school_id.2 - deleted
status_college_id.1 - deleted
status_college_id.2 - deleted
status_display.1 - deleted
status_display.2 - deleted
russian_level_id.1 - deleted
russian_level_id.2 - deleted
home_country_id.1 - deleted
home_country_id.2 - deleted
information.1 - deleted
information.2 - deleted
is_confirmed.1 - deleted
is_confirmed.2 - deleted


In [ ]:
#t_info2.info()

In [33]:
pd.set_option('display.max_columns', None)

In [34]:
t_info2.head(5)

,working_teacher_id,is_home_lessons,is_external_lessons,external_comments,lesson_duration,lesson_cost,status_id,status_school_id,status_college_id,status_display,russian_level_id,home_country_id,information,is_confirmed,status_relevant_date.1,photo_path,is_display,rating_id,rating,comments,is_pupils_needed,is_cell_phone_confirmed,effective_rating,area_id,pupil_needed_date,sex,amount_to_pay,is_remote_lessons,remote_comments,show_on_map,send_mailing,send_suitable_orders,rating_for_users,rating_for_admin,passport_id,is_edited,orders_allowed,display_days,verification_status_id,is_individual,partner_id,star_rating,rating_for_users_yesterday,review_num,relevance_date,is_display_at_partners,status_institution_id,free_time_relevance_date,age,expirience
0,45,0,0,NaN,60,1000,4,NaN,NaN,0,5,7,[p]&nbsp;[/p][p]&nbsp;[/p][p]?[/p][p]133320 Ка...,1,NaN,/teachers/33/photo/photo_small.jpg,1,15,5,Швачко М.Н. 15.01.2018 14:59 - 33\r\nАникеева ...,1.0,0,4.679688,1,NaN,1,500,1,NaN,0,1,0,1138.0,237.5,NaN,1,0,492.0,1,NaN,NaN,4.500000,1138.0,6,2022-05-16 18:44:59.993,0,3702.0,2023-11-06 14:20:15.173,40.453692,34.207117
1,63,1,1,NaN,60,500,5,NaN,NaN,1,5,7,NaN,1,NaN,/teachers/46/photo/photo_small.webp,0,4,-3,NaN,1.0,0,0.000000,1,NaN,3,0,0,NaN,0,1,1,25.0,0.0,NaN,1,1,0.0,1,NaN,NaN,4.300781,25.0,0,2017-01-01 00:00:00.000,1,NaN,NaN,42.966394,19.399271
2,68,0,0,NaN,60,1500,5,NaN,NaN,1,5,7,[p]Готовлю к ЕГЭ и ОГЭ более 20 лет. Мои учени...,1,NaN,/teachers/51/photo/photo_small.webp,0,16,6,NaN,1.0,0,6.000000,1,NaN,2,0,1,Только удаленные занятия по Skype.,0,1,0,300.0,0.0,NaN,1,1,2182.0,3,0.0,NaN,4.300781,300.0,0,2017-01-01 00:00:00.000,0,NaN,NaN,58.107886,34.751721
3,69,0,1,Ближайший район: м. Алтуфьево.,60,1000,5,NaN,NaN,1,5,7,NaN,1,NaN,/teachers/52/photo/photo_small.webp,1,16,6,NaN,0.0,0,11.289062,1,2019-12-31 00:00:00.000,2,0,1,NaN,0,1,1,135.0,0.0,NaN,1,1,2084.0,1,0.0,NaN,5.000000,135.0,2,2020-04-01 01:07:44.157,1,NaN,NaN,34.562860,16.395737
4,70,0,1,Выезд в удобное для Вас время по Москве.,45,400,5,NaN,NaN,1,5,7,"4 года репетиторской деятельности, \r\nподгото...",1,NaN,/teachers/54/photo/photo_small.webp,0,4,-3,"21,01,2008 Не очень хорошо повела себя при на ...",1.0,0,-5.000000,1,NaN,3,0,0,NaN,0,1,1,50.0,0.0,NaN,0,1,0.0,1,NaN,NaN,4.300781,50.0,0,2017-01-01 00:00:00.000,1,NaN,NaN,37.330648,19.064895


In [35]:
t_info2[['external_comments', 'information','comments', 'remote_comments']] =  \
t_info2[['external_comments', 'information','comments', 'remote_comments']].astype(np.object).fillna(0)

/tmp/ipykernel_10969/1631202341.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  t_info2[['external_comments', 'information','comments', 'remote_comments']].astype(np.object).fillna(0)


In [36]:
#t_info2.isna().sum()

## Объединим таблицы с информацией об учителях

In [44]:
new_t_info = t_info2.merge(suit_t, how='right')

In [45]:
new_t_info.head()

,working_teacher_id,is_home_lessons,is_external_lessons,external_comments,lesson_duration,lesson_cost,status_id,status_school_id,status_college_id,status_display,russian_level_id,home_country_id,information,is_confirmed,status_relevant_date.1,photo_path,is_display,rating_id,rating,comments,is_pupils_needed,is_cell_phone_confirmed,effective_rating,area_id,pupil_needed_date,sex,amount_to_pay,is_remote_lessons,remote_comments,show_on_map,send_mailing,send_suitable_orders,rating_for_users,rating_for_admin,passport_id,is_edited,orders_allowed,display_days,verification_status_id,is_individual,partner_id,star_rating,rating_for_users_yesterday,review_num,relevance_date,is_display_at_partners,status_institution_id,free_time_relevance_date,age,expirience,id,contact_result,assign
0,82583,1.0,1.0,0,60.0,1200.0,5.0,NaN,NaN,1.0,5.0,7.0,[p]Являюсь дипломированным преподавателем англ...,1.0,NaN,/teachers/81285/photo/photo_small.webp,1.0,17.0,7.0,0,NaN,0.0,0.0,1.0,2023-12-29 00:00:00.000,2.0,0.0,1.0,Проводит занятия по скайпу. Стоимость 500 рубл...,1.0,0.0,1.0,1191.0,882.0,NaN,1.0,1.0,3020.0,3.0,0.0,NaN,5.000000,1191.0,6.0,2023-08-31 10:39:43.377,1.0,NaN,NaN,40.124505,11.003957,1742711,Репетитор согласился,2
1,125057,1.0,1.0,0,60.0,700.0,5.0,NaN,NaN,0.0,5.0,7.0,0,1.0,NaN,/teachers/103413/photo/photo_small.webp,1.0,6.0,-5.0,0,NaN,0.0,0.0,1.0,NaN,2.0,0.0,0.0,0,1.0,1.0,1.0,210.0,0.0,NaN,1.0,1.0,1392.0,3.0,0.0,NaN,5.000000,210.0,2.0,2017-10-16 19:23:01.000,1.0,NaN,NaN,25.964007,8.388665,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
2,231211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
3,128773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
4,63019,1.0,1.0,0,90.0,1300.0,5.0,NaN,NaN,0.0,0.0,7.0,"Индивидуальный подход к ученикам, нестандартны...",1.0,NaN,/teachers/61846/photo/photo_small.webp,0.0,6.0,-5.0,0,NaN,0.0,9.0,1.0,NaN,2.0,0.0,1.0,0,0.0,1.0,1.0,50.0,0.0,NaN,1.0,1.0,532.5,0.0,NaN,NaN,4.300781,50.0,0.0,2017-01-01 00:00:00.000,1.0,NaN,NaN,33.031976,11.393620,1742713,Репетитор согласился,1


In [47]:
new_t_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7446818 entries, 0 to 7446817
Data columns (total 53 columns):
 #   Column                      Dtype   
---  ------                      -----   
 0   working_teacher_id          int32   
 1   is_home_lessons             float64 
 2   is_external_lessons         float64 
 3   external_comments           object  
 4   lesson_duration             float64 
 5   lesson_cost                 float64 
 6   status_id                   float64 
 7   status_school_id            float16 
 8   status_college_id           float16 
 9   status_display              float64 
 10  russian_level_id            float64 
 11  home_country_id             float64 
 12  information                 object  
 13  is_confirmed                float64 
 14  status_relevant_date.1      category
 15  photo_path                  category
 16  is_display                  float64 
 17  rating_id                   float64 
 18  rating                      float64 
 19  

Удалим неинформативные колонки и заполним пропуски в данных

In [48]:
new_t_info = new_t_info.drop(columns=['status_relevant_date.1', 'pupil_needed_date', 'partner_id', 'relevance_date', 'free_time_relevance_date'])

In [49]:
new_t_info['photo_path'] = new_t_info['photo_path'].astype('object')

In [50]:
new_t_info = new_t_info.fillna(0)

In [194]:
new_t_info.head(3)

,working_teacher_id,is_home_lessons,is_external_lessons,external_comments,lesson_duration,lesson_cost,status_id,status_school_id,status_college_id,status_display,russian_level_id,home_country_id,information,is_confirmed,photo_path,is_display,rating_id,rating,comments,is_pupils_needed,is_cell_phone_confirmed,effective_rating,area_id,sex,amount_to_pay,is_remote_lessons,remote_comments,show_on_map,send_mailing,send_suitable_orders,rating_for_users,rating_for_admin,passport_id,is_edited,orders_allowed,display_days,verification_status_id,is_individual,star_rating,rating_for_users_yesterday,review_num,is_display_at_partners,status_institution_id,age,expirience,id,contact_result,assign
0,82583,1.0,1.0,0,60.0,1200.0,5.0,0.0,0.0,1.0,5.0,7.0,[p]Являюсь дипломированным преподавателем англ...,1.0,/teachers/81285/photo/photo_small.webp,1.0,17.0,7.0,0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,Проводит занятия по скайпу. Стоимость 500 рубл...,1.0,0.0,1.0,1191.0,882.0,0.0,1.0,1.0,3020.0,3.0,0.0,5.0,1191.0,6.0,1.0,0.0,40.124505,11.003957,1742711,Репетитор согласился,2
1,125057,1.0,1.0,0,60.0,700.0,5.0,0.0,0.0,0.0,5.0,7.0,0,1.0,/teachers/103413/photo/photo_small.webp,1.0,6.0,-5.0,0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0,1.0,1.0,1.0,210.0,0.0,0.0,1.0,1.0,1392.0,3.0,0.0,5.0,210.0,2.0,1.0,0.0,25.964007,8.388665,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
2,231211,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0


## Посчитаем процент нулей в столбцах таблицы об учителях

In [195]:
len(new_t_info)

7446818

In [196]:
count = ((new_t_info == 0).sum()/len(new_t_info)*100).sort_values(ascending=False)

In [201]:
count = count.reset_index()

### Удалим столбцы, где более 90 процентов 0 значений

In [202]:
columns_to_dell = []
for i in range(len(count)):
    if count.iloc[i][0]>90:
       columns_to_dell.append(count.iloc[i]['index'])
columns_to_dell

['amount_to_pay',
 'passport_id',
 'is_individual',
 'is_cell_phone_confirmed',
 'status_college_id',
 'remote_comments',
 'is_pupils_needed',
 'status_school_id',
 'rating_for_admin',
 'status_institution_id',
 'comments']

In [203]:
new_t_info_dell= new_t_info.drop(columns=columns_to_dell)

In [204]:
new_t_info_dell.head(2)

,working_teacher_id,is_home_lessons,is_external_lessons,external_comments,lesson_duration,lesson_cost,status_id,status_display,russian_level_id,home_country_id,information,is_confirmed,photo_path,is_display,rating_id,rating,effective_rating,area_id,sex,is_remote_lessons,show_on_map,send_mailing,send_suitable_orders,rating_for_users,is_edited,orders_allowed,display_days,verification_status_id,star_rating,rating_for_users_yesterday,review_num,is_display_at_partners,age,expirience,id,contact_result,assign
0,82583,1.0,1.0,0,60.0,1200.0,5.0,1.0,5.0,7.0,[p]Являюсь дипломированным преподавателем англ...,1.0,/teachers/81285/photo/photo_small.webp,1.0,17.0,7.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1191.0,1.0,1.0,3020.0,3.0,5.0,1191.0,6.0,1.0,40.124505,11.003957,1742711,Репетитор согласился,2
1,125057,1.0,1.0,0,60.0,700.0,5.0,0.0,5.0,7.0,0,1.0,/teachers/103413/photo/photo_small.webp,1.0,6.0,-5.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0,210.0,1.0,1.0,1392.0,3.0,5.0,210.0,2.0,1.0,25.964007,8.388665,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0


### Заменим на 1 информацию в столбцах 'photo_path', 'information'

In [209]:
new_t_info_dell.loc[(new_t_info_dell['photo_path'] != 0) | (new_t_info_dell['information'] != 0), ['photo_path', 'information']] = 1 
new_t_info_dell.head()

,working_teacher_id,is_home_lessons,is_external_lessons,external_comments,lesson_duration,lesson_cost,status_id,status_display,russian_level_id,home_country_id,information,is_confirmed,photo_path,is_display,rating_id,rating,effective_rating,area_id,sex,is_remote_lessons,show_on_map,send_mailing,send_suitable_orders,rating_for_users,is_edited,orders_allowed,display_days,verification_status_id,star_rating,rating_for_users_yesterday,review_num,is_display_at_partners,age,expirience,id,contact_result,assign
0,82583,1.0,1.0,0,60.0,1200.0,5.0,1.0,5.0,7.0,1,1.0,1,1.0,17.0,7.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1191.0,1.0,1.0,3020.0,3.0,5.000000,1191.0,6.0,1.0,40.124505,11.003957,1742711,Репетитор согласился,2
1,125057,1.0,1.0,0,60.0,700.0,5.0,0.0,5.0,7.0,1,1.0,1,1.0,6.0,-5.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0,210.0,1.0,1.0,1392.0,3.0,5.000000,210.0,2.0,1.0,25.964007,8.388665,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
2,231211,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
3,128773,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1742713,"Репетитор положил трубку, либо обрыв связи (Ok...",0
4,63019,1.0,1.0,0,90.0,1300.0,5.0,0.0,0.0,7.0,1,1.0,1,0.0,6.0,-5.0,9.0,1.0,2.0,1.0,0.0,1.0,1.0,50.0,1.0,1.0,532.5,0.0,4.300781,50.0,0.0,1.0,33.031976,11.393620,1742713,Репетитор согласился,1


### В столбце 'contact_result' найдем слово 'согласился'

Педварительно приведем текст к нижнему регистру и очистим его

In [213]:
new_t_info_dell['contact_result'] = new_t_info_dell['contact_result'].str.lower()

In [218]:
def find4_text(text):
    lemm=0
    if re.findall(r'соглас|$', str(text))[0]:
        lemm = 1
    return lemm

In [219]:
new_t_info_dell['contact_result_new'] = new_t_info_dell['contact_result'].progress_apply(find4_text)

progress: 100%|██████████| 7446818/7446818 [00:49<00:00, 150371.22it/s]


In [221]:
new_t_info_dell

,working_teacher_id,is_home_lessons,is_external_lessons,external_comments,lesson_duration,lesson_cost,status_id,status_display,russian_level_id,home_country_id,information,is_confirmed,photo_path,is_display,rating_id,rating,effective_rating,area_id,sex,is_remote_lessons,show_on_map,send_mailing,send_suitable_orders,rating_for_users,is_edited,orders_allowed,display_days,verification_status_id,star_rating,rating_for_users_yesterday,review_num,is_display_at_partners,age,expirience,id,contact_result,assign,contact_result_edit,contact_result_new
0,82583,1.0,1.0,0,60.0,1200.0,5.0,1.0,5.0,7.0,1,1.0,1,1.0,17.0,7.0,0.0,1.0,2.0,1.0,1.0,0.0,1.0,1191.0,1.0,1.0,3020.0,3.0,5.000000,1191.0,6.0,1.0,40.124505,11.003957,1742711,репетитор согласился,2,соглас,1
1,125057,1.0,1.0,0,60.0,700.0,5.0,0.0,5.0,7.0,1,1.0,1,1.0,6.0,-5.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0,210.0,1.0,1.0,1392.0,3.0,5.000000,210.0,2.0,1.0,25.964007,8.388665,1742713,"репетитор положил трубку, либо обрыв связи (ok...",0,,0
2,231211,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1742713,"репетитор положил трубку, либо обрыв связи (ok...",0,,0
3,128773,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,1742713,"репетитор положил трубку, либо обрыв связи (ok...",0,,0
4,63019,1.0,1.0,0,90.0,1300.0,5.0,0.0,0.0,7.0,1,1.0,1,0.0,6.0,-5.0,9.0,1.0,2.0,1.0,0.0,1.0,1.0,50.0,1.0,1.0,532.5,0.0,4.300781,50.0,0.0,1.0,33.031976,11.393620,1742713,репетитор согласился,1,соглас,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7446813,175947,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,2944965,репетитор согласился,2,соглас,1
7446814,197299,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,2946576,",",0,,0
7446815,152334,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,2946689,репетитор согласился,1,соглас,1
7446816,139143,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,2946689,репетитор согласился,1,соглас,1


### Сгруппируем данные по номеру заказа следующим образом:

In [225]:
g = {'is_home_lessons':['count'], 
     'is_external_lessons':['count'],
     'lesson_duration':['mean'],
     'lesson_cost':['mean'],
     'rating': ['mean'],
     'sex':['sum'],
     'is_remote_lessons':['count'],
     'rating_for_users':['mean'],
     'review_num':['mean'],
     'age':['mean'],
     'expirience':['mean'],
     'assign':['count'],
     'contact_result_new': ['sum'],
    'working_teacher_id' :['count']
    }

teachers1 = new_t_info_dell.groupby(['id']).agg(g).reset_index()

In [232]:
teachers2 = teachers1.droplevel(1, axis=1)

In [230]:
teachers2.rename(columns={'working_teacher_id':'number_of_teachers'}, inplace = True )

In [231]:
teachers2

,id,is_home_lessons,is_external_lessons,lesson_duration,lesson_cost,rating,sex,is_remote_lessons,rating_for_users,review_num,age,expirience,assign,contact_result_new,number_of_teachers
0,1742638,1,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.000000,0.000000,1,0,1
1,1742639,1,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.000000,0.000000,1,1,1
2,1742650,2,2,30.0,200.0,-0.5,2.0,2,72.5,2.0,16.841700,7.196495,2,1,2
3,1742655,1,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.000000,0.000000,1,1,1
4,1742661,1,1,60.0,2000.0,6.0,2.0,1,543.5,1.0,37.678326,16.524901,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676712,2946663,1,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.000000,0.000000,1,1,1
676713,2946677,1,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.000000,0.000000,1,1,1
676714,2946678,1,1,0.0,0.0,0.0,0.0,1,0.0,0.0,0.000000,0.000000,1,1,1
676715,2946689,2,2,0.0,0.0,0.0,0.0,2,0.0,0.0,0.000000,0.000000,2,2,2


### Таблица готова для объединения с таблицей orders.feather

Сохраним таблицу в формате cvs/feather

In [233]:
teachers2.to_csv('teachers_ready.csv', index=False)